**Database queries documentation will be included prior to each query**


In [2]:
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, Date, Float
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, Date, Float, func


Base = declarative_base()



C:\Users\haulmark\AppData\Local\Temp\ipykernel_46096\1344665909.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
# Categories Table
class Category(Base):
    __tablename__ = 'Categories'
    
    CategoryID = Column(Integer, primary_key=True, autoincrement=True, extend_existing=True)
    CategoryName = Column(String, extend_existing=True)


# Users Table
class User(Base):
    __tablename__ = 'Users'
    
    UserID = Column(Integer, primary_key=True, autoincrement=True)
    FirstName = Column(String)
    LastName = Column(String)
    Email = Column(String)
    JoinDate = Column(Date)

# Recipes Table
class Recipe(Base):
    __tablename__ = 'Recipes'
    
    RecipeID = Column(Integer, primary_key=True, autoincrement=True)
    Title = Column(String)
    Description = Column(String)
    CookingTime = Column(Integer)
    Servings = Column(Integer)
    NumberOfSteps = Column(Integer)
    SubmittedDate = Column(Date)
    NumberOfIngredients = Column(Integer)
    ImageURL = Column(String)

# Ingredients Table
class Ingredient(Base):
    __tablename__ = 'Ingredients'
    
    IngredientsID = Column(Integer, primary_key=True, autoincrement=True)
    Ingredients = Column(String, nullable=False)
    ServingSize = Column(Integer)
    Calories = Column(Integer)
    TotalFat = Column(Integer)
    SaturatedFat = Column(Integer)
    Cholesterol = Column(Integer)
    Sodium = Column(Integer)
    Carbohydrate = Column(Integer)
    Fiber = Column(Integer)
    Sugars = Column(Integer)
    Protein = Column(Integer)
    Fat = Column(Integer)
    Caffeine = Column(Integer)
    Source = Column(String)
    Date = Column(String)
    IsAllergen = Column(String)
    Food_allergen_ID = Column(Integer)
    Food = Column(String)
    Food_Class = Column(String)
    Food_type = Column(String)
    Food_Group = Column(String)
    Allergy_Type = Column(String)

# Recipe_Ingredients Fact Table
class RecipeIngredientsFact(Base):
    __tablename__ = 'Recipe_Ingredients_fact_table'
    
    CompID = Column(Integer, primary_key=True, autoincrement=True)
    RecipeID = Column(Integer, ForeignKey('recipes.RecipeID'))
    IngredientsID = Column(Integer, ForeignKey('ingredients.IngredientsID'))
    Ingredients = Column(String)

# Reviews Table
class Review(Base):
    __tablename__ = 'Reviews'
    
    ReviewID = Column(Integer, primary_key=True, autoincrement=True)
    Rating = Column(Integer)
    ReviewText = Column(String)
    ReviewDate = Column(Date)
    UserID = Column(Integer, ForeignKey('users.UserID'))
    RecipeID = Column(Integer, ForeignKey('recipes.RecipeID'))

# Recipe_Categories Table
class RecipeCategory(Base):
    __tablename__ = 'Recipe_Categories'
    
    CategoryID = Column(Integer, ForeignKey('categories.CategoryID'), primary_key=True)
    RecipeID = Column(Integer, ForeignKey('recipes.RecipeID'), primary_key=True)

class CategoryRecipeFact(Base):
    __tablename__ = 'Category_Recipe_fact_table'
    
    # Add a unique primary key column
    CompID = Column(Integer, primary_key=True, autoincrement=True)
    
    # Foreign key relationships
    CategoryID = Column(Integer, ForeignKey('Categories.CategoryID'))
    RecipeID = Column(Integer, ForeignKey('Recipes.RecipeID'))


    

C:\Users\haulmark\AppData\Local\Temp\ipykernel_46096\1883591324.py:5: SAWarning: Can't validate argument 'extend_existing'; can't locate any SQLAlchemy dialect named 'extend'
  CategoryID = Column(Integer, primary_key=True, autoincrement=True, extend_existing=True)
C:\Users\haulmark\AppData\Local\Temp\ipykernel_46096\1883591324.py:6: SAWarning: Can't validate argument 'extend_existing'; can't locate any SQLAlchemy dialect named 'extend'
  CategoryName = Column(String, extend_existing=True)


In [4]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Assuming my_recipes.db is in the root and Queries is the folder where the query file is.
engine = create_engine('sqlite:///../my_recipes.db')

# Create a session to interact with the existing database
Session = sessionmaker(bind=engine)
session = Session()

print("Connected to the existing database.")



Connected to the existing database.



**Query 1: Recipe Ingredients Query with Dynamic Input**

**Query Purpose**: This query retrieves a list of recipes along with their ingredients based on a user-specified category. It dynamically accepts a category name as input, making it flexible for users to filter results by category. If the category does not exist, it prompts the user with a list of available categories.

**Technical Details**:

- **Tables Involved**:
  - `Recipes`: Holds recipe details.
  - `Recipe_Ingredients_fact_table`: Links recipes and ingredients.
  - `Ingredients`: Contains ingredient details.
  - `Categories`: Contains category information.
- **Join Operations**: The query performs a JOIN across multiple tables (`Recipes`, `Recipe_Ingredients_fact_table`, `Ingredients`) to retrieve the relevant data.
- **Dynamic Input**: The category is provided dynamically, and error handling ensures that if the category doesn’t exist or has no associated recipes, the user is prompted to try another category.
- **Output**: The query returns a list of recipes with their respective ingredients, filtered by the chosen category.

In [47]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Assuming you have already defined your ORM classes like Recipe, Ingredient, etc.
# Set up your SQLite database connection
engine = create_engine(r"sqlite:///P:/MABA/Seminar/recipes database/my_recipes.db")  # Update with your actual path
Session = sessionmaker(bind=engine)
session = Session()

# Step 1: Display a list of available categories
def display_categories():
    available_categories = session.query(Category.CategoryID, Category.CategoryName).all()
    
    print("Here are our available recipe categories:")
    for index, category in enumerate(available_categories, 1):
        print(f"{index}. {category.CategoryName}")
    
    return available_categories

# Step 2: Prompt the user to select a category
def select_category(available_categories):
    print("\nPlease wait, displaying available categories...\n")
    
    selected_index = None
    while selected_index is None:
        try:
            selected_index = int(input("Please select a category by entering its number: ")) - 1
            if 0 <= selected_index < len(available_categories):
                return available_categories[selected_index]
            else:
                print("Invalid selection. Please try again.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

# Step 3: Display favorite recipes from the selected category
def display_recipes_by_category(category_id):
    recipes = session.query(Recipe.RecipeID, Recipe.Title).join(
        CategoryRecipeFact, Recipe.RecipeID == CategoryRecipeFact.RecipeID
    ).filter(CategoryRecipeFact.CategoryID == category_id).limit(10).all()

    if not recipes:
        print("No recipes found for the selected category.")
        return []

    print("\nGreat choice! Here is a sample of some user favorites from that category:")
    for index, recipe in enumerate(recipes, 1):
        print(f"{index}. {recipe.Title}")

    return recipes

# Step 4: Prompt the user to select a recipe and display details
def get_recipe_details(recipe_id):
    print(f"Fetching details for recipe ID: {recipe_id}")  # Debugging

    # Fetch recipe details from the Recipe table
    recipe = session.query(Recipe).filter(Recipe.RecipeID == recipe_id).first()

    if recipe is None:
        print(f"No recipe found with ID: {recipe_id}")
        return

    print(f"Recipe details found for {recipe_id}: {recipe.Title}")  # Debugging

    # Fetch ingredients for the selected recipe
    ingredients = session.query(Ingredient).join(
        RecipeIngredientsFact, Ingredient.IngredientsID == RecipeIngredientsFact.IngredientsID
    ).filter(RecipeIngredientsFact.RecipeID == recipe_id).all()

    # Display recipe details
    print(f"\nRecipe: {recipe.Title}")
    print(f"Description: {recipe.Description}")
    print(f"Cooking Time: {recipe.CookingTime} minutes")
    
       # Display ingredients
    print("Ingredients:")
    for ingredient in ingredients:
        print(f"- {ingredient.Ingredients}")

# Main function to drive the program
def main():
    available_categories = display_categories()

    if available_categories:
        category = select_category(available_categories)
        if category:
            recipes = display_recipes_by_category(category.CategoryID)
            if recipes:
                select_and_display_recipe(recipes)
            else:
                print("No recipes found in the selected category.")
        else:
            print("No valid category selected.")
    else:
        print("No categories available.")

# Run the program
if __name__ == "__main__":
    main()


Here are our available recipe categories:
1. Snacks
2. Soups
3. Desserts
4. Main Courses
5. Breads
6. Salads
7. Appetizers
8. Poultry
9. Sauces
10. Seafood
11. Beverages
12. Pork
13. Beef
14. Lamb
15. Vegetarian
16. Pasta
17. Vegan

Please wait, displaying available categories...


Great choice! Here is a sample of some user favorites from that category:
1. Winter Squash Apple And Walnut Soup
2. Steph S Chicken Tortilla Soup
3. Zupa Ze Swiezych Grzybow Polish Mushroom Soup
4. The Bomb Pumpkin Soup
5. 10 Minute Cream Of Mushroom Soup
6. Amazing French Onion Soup
7. Anything Goes Seafood Chowder
8. Asparagus Brie Soup
9. Bacon And Baked Potato Soup
10. Best Cream Of Broccoli Soup

Please choose from the favorite recipes listed below:
1. Winter Squash Apple And Walnut Soup
2. Steph S Chicken Tortilla Soup
3. Zupa Ze Swiezych Grzybow Polish Mushroom Soup
4. The Bomb Pumpkin Soup
5. 10 Minute Cream Of Mushroom Soup
6. Amazing French Onion Soup
7. Anything Goes Seafood Chowder
8. Asparagus B

**Query 2: Reviews Query with Parameterized Input**

**Query Purpose**: This query retrieves reviews for a specific recipe, using a parameterized input for recipe ID. This is useful for retrieving all reviews associated with a particular recipe based on user input.

**Technical Details**:

- **Tables Involved**:
  - `Reviews`: Holds reviews with ratings, review text, and associated recipe IDs.
  - `Recipes`: Links the reviews to the corresponding recipes.
- **Join Operation**: The query performs a JOIN between `Reviews` and `Recipes` using the `RecipeID` to ensure that reviews are linked to the correct recipe.
- **Parameterized Input**: The query takes a recipe ID as an input parameter, ensuring that the data retrieved is specific to a particular recipe.
- **Output**: The query returns a list of reviews, including ratings and review text, for the specified recipe.

In [6]:
# Define a function to get reviews for a specific recipe
def get_reviews_for_recipe(recipe_id):
    # Query the Review table for reviews matching the given RecipeID
    return session.query(Review).filter(Review.RecipeID == recipe_id).all()

# Call the function to find a recipe with reviews
recipe_with_reviews = session.query(Recipe.RecipeID).select_from(Recipe).join(Review, Recipe.RecipeID == Review.RecipeID).first()

# If a recipe with reviews exists, query its reviews
if recipe_with_reviews:
    recipe_id = recipe_with_reviews.RecipeID
    reviews = get_reviews_for_recipe(recipe_id)

    if reviews:
        for review in reviews:
            print(f"Review: {review.ReviewText}, Rating: {review.Rating}")
    else:
        print(f"No reviews found for RecipeID {recipe_id}")
else:
    print("No recipes with reviews found.")




Review: This recipe is okay., Rating: 3


**Query 3 Average Rating Aggregation Query**

**Query Purpose**: This query calculates the average rating for each recipe. It provides a summary of how users have rated different recipes, which is useful for understanding overall user satisfaction.

**Technical Details**:

- **Tables Involved**:
  - `Reviews`: Contains the ratings for each recipe.
  - `Recipes`: Provides recipe details linked to the reviews.
- **Aggregation**: The query uses the SQL `AVG()` function to calculate the average rating of each recipe. This is grouped by the `RecipeID` to ensure that the average is calculated on a per-recipe basis.
- **Group By**: The query groups results by `RecipeID`, providing one average rating per recipe.
- **Output**: It returns the recipe title and the corresponding average rating, providing a high-level overview of user ratings across recipes.

In [7]:
# Query 3 function to get the average rating for each recipe (Aggregated Data)
def get_average_rating_per_recipe():
    return session.query(Recipe.RecipeID, Recipe.Title, func.avg(Review.Rating).label('average_rating')).join(
        Review, Recipe.RecipeID == Review.RecipeID).group_by(Recipe.RecipeID).all()

# Query 3 call to get the average rating for each recipe
average_ratings = get_average_rating_per_recipe()
for recipe_id, title, avg_rating in average_ratings:
    print(f"Recipe: {title}, Average Rating: {avg_rating}")


Recipe: Squash And Kale Risotto, Average Rating: 3.0
Recipe: Winter Squash Apple And Walnut Soup, Average Rating: 4.0
Recipe: Wonderful Winter Squash Risotto, Average Rating: 5.0
Recipe: Steph S Chicken Tortilla Soup, Average Rating: 1.0
Recipe: Cumberland Rum Butter, Average Rating: 5.0


**Query 4 Return Recipes with less than 10 inrgredients**

**Query Purpose**: This query recipe titles with less than 10 ingredients

**Technical Details**:

- **Tables Involved**:
  - `Recipes`: Provides recipe details linked to the reviews.

- **Output**: It returns the recipe title and the corresponding number of ingredients

In [8]:
from sqlalchemy import cast, String
from datetime import datetime

# Function to convert date strings to ISO format (YYYY-MM-DD)
def convert_to_iso_date(date_string):
    try:
        # Try to parse the date in 'MM/DD/YYYY' format
        date_object = datetime.strptime(date_string, '%m/%d/%Y')
        # Convert it to 'YYYY-MM-DD' format
        return date_object.strftime('%Y-%m-%d')
    except ValueError:
        # If parsing fails, return the original string
        return date_string

# Function to get recipes with less than 10 ingredients, limited to 10 results
def get_recipes_with_few_ingredients():
    # Query to return recipes where the number of ingredients is less than 10, limited to 10
    recipes = session.query(
        Recipe.RecipeID, Recipe.Title, Recipe.NumberOfIngredients, 
        cast(Recipe.SubmittedDate, String)  # Ensure date is treated as a string
    ).filter(Recipe.NumberOfIngredients < 10).limit(10).all()
    
    # Manually convert the date format for each recipe
    for recipe in recipes:
        recipe_date = recipe[3]  # Get the SubmittedDate from the tuple
        formatted_date = convert_to_iso_date(recipe_date)
        recipe = recipe[:3] + (formatted_date,)  # Update the tuple with formatted date
    
    return recipes

# Function to display the recipes
def display_recipes_with_few_ingredients():
    recipes = get_recipes_with_few_ingredients()
    
    if recipes:
        print(f"Recipes with fewer than 10 ingredients (showing up to 10):")
        for recipe in recipes:
            print(f"Recipe: {recipe[1]}, Ingredients Count: {recipe[2]}, Submitted Date: {recipe[3]}")
    else:
        print("No recipes found with fewer than 10 ingredients.")

# Call the function to display the recipes
display_recipes_with_few_ingredients()


Recipes with fewer than 10 ingredients (showing up to 10):
Recipe: Arriba Baked Winter Squash Mexican Style, Ingredients Count: 7, Submitted Date: 9/16/2005
Recipe: Futari, Ingredients Count: 8, Submitted Date: 10/10/2005
Recipe: Jalapeno Poppers Stuffed With Squash And Feta, Ingredients Count: 6, Submitted Date: 7/28/2003
Recipe: Orange Winter Squash Casserole, Ingredients Count: 7, Submitted Date: 11/18/2003
Recipe: Wonderful Winter Squash Risotto, Ingredients Count: 7, Submitted Date: 11/28/2003
Recipe: Steph S Chicken Tortilla Soup, Ingredients Count: 9, Submitted Date: 10/16/2001
Recipe: Cumberland Rum Butter, Ingredients Count: 4, Submitted Date: 3/21/2002
Recipe: Eggless Apple Cake, Ingredients Count: 7, Submitted Date: 5/8/2002
Recipe: Simplest Cake In The World, Ingredients Count: 5, Submitted Date: 1/3/2003
Recipe: I Wanna Live With A Cinnamon Girl Honey, Ingredients Count: 2, Submitted Date: 12/27/2004


Version testing for Query for returning a recipe from a category

In [46]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Assuming you have already defined your ORM classes like Recipe, Ingredient, etc.
# Set up your SQLite database connection
engine = create_engine(r"sqlite:///P:/MABA/Seminar/recipes database/my_recipes.db")  # Update with your actual path
Session = sessionmaker(bind=engine)
session = Session()

# Step 1: Display a list of available categories
def display_categories():
    available_categories = session.query(Category.CategoryID, Category.CategoryName).all()
    
    print("Here are our available recipe categories:")
    for index, category in enumerate(available_categories, 1):
        print(f"{index}. {category.CategoryName}")
    
    return available_categories

# Step 2: Prompt the user to select a category
def select_category(available_categories):
    print("\nPlease wait, displaying available categories...\n")
    
    selected_index = None
    while selected_index is None:
        try:
            selected_index = int(input("Please select a category by entering its number: ")) - 1
            if 0 <= selected_index < len(available_categories):
                return available_categories[selected_index]
            else:
                print("Invalid selection. Please try again.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

# Step 3: Display favorite recipes from the selected category
def display_recipes_by_category(category_id):
    recipes = session.query(Recipe.RecipeID, Recipe.Title).join(
        CategoryRecipeFact, Recipe.RecipeID == CategoryRecipeFact.RecipeID
    ).filter(CategoryRecipeFact.CategoryID == category_id).limit(10).all()

    if not recipes:
        print("No recipes found for the selected category.")
        return []

    print("\nGreat choice! Here is a sample of some user favorites from that category:")
    for index, recipe in enumerate(recipes, 1):
        print(f"{index}. {recipe.Title}")

    return recipes

# Step 4: Prompt the user to select a recipe and display details
def get_recipe_details(recipe_id):
    print(f"Fetching details for recipe ID: {recipe_id}")  # Debugging

    # Fetch recipe details from the Recipe table
    recipe = session.query(Recipe).filter(Recipe.RecipeID == recipe_id).first()

    if recipe is None:
        print(f"No recipe found with ID: {recipe_id}")
        return

    print(f"Recipe details found for {recipe_id}: {recipe.Title}")  # Debugging

    # Fetch ingredients for the selected recipe
    ingredients = session.query(Ingredient).join(
        RecipeIngredientsFact, Ingredient.IngredientsID == RecipeIngredientsFact.IngredientsID
    ).filter(RecipeIngredientsFact.RecipeID == recipe_id).all()

    # Display recipe details
    print(f"\nRecipe: {recipe.Title}")
    print(f"Description: {recipe.Description}")
    print(f"Cooking Time: {recipe.CookingTime} minutes")
    
       # Display ingredients
    print("Ingredients:")
    for ingredient in ingredients:
        print(f"- {ingredient.Ingredients}")

# Main function to drive the program
def main():
    available_categories = display_categories()

    if available_categories:
        category = select_category(available_categories)
        if category:
            recipes = display_recipes_by_category(category.CategoryID)
            if recipes:
                select_and_display_recipe(recipes)
            else:
                print("No recipes found in the selected category.")
        else:
            print("No valid category selected.")
    else:
        print("No categories available.")

# Run the program
if __name__ == "__main__":
    main()


Here are our available recipe categories:
1. Snacks
2. Soups
3. Desserts
4. Main Courses
5. Breads
6. Salads
7. Appetizers
8. Poultry
9. Sauces
10. Seafood
11. Beverages
12. Pork
13. Beef
14. Lamb
15. Vegetarian
16. Pasta
17. Vegan

Please wait, displaying available categories...


Great choice! Here is a sample of some user favorites from that category:
1. Winter Squash Apple And Walnut Soup
2. Steph S Chicken Tortilla Soup
3. Zupa Ze Swiezych Grzybow Polish Mushroom Soup
4. The Bomb Pumpkin Soup
5. 10 Minute Cream Of Mushroom Soup
6. Amazing French Onion Soup
7. Anything Goes Seafood Chowder
8. Asparagus Brie Soup
9. Bacon And Baked Potato Soup
10. Best Cream Of Broccoli Soup

Please choose from the favorite recipes listed below:
1. Winter Squash Apple And Walnut Soup
2. Steph S Chicken Tortilla Soup
3. Zupa Ze Swiezych Grzybow Polish Mushroom Soup
4. The Bomb Pumpkin Soup
5. 10 Minute Cream Of Mushroom Soup
6. Amazing French Onion Soup
7. Anything Goes Seafood Chowder
8. Asparagus B

testing query v2

In [45]:
from datetime import datetime

def get_recipe_details(recipe_id):
    print(f"Fetching details for recipe ID: {recipe_id}")  # Debugging

    # Fetch recipe details from the Recipe table
    recipe = session.query(Recipe).filter(Recipe.RecipeID == recipe_id).first()

    if recipe is None:
        print(f"No recipe found with ID: {recipe_id}")
        return

    print(f"Recipe details found for {recipe_id}: {recipe.Title}")  # Debugging

    # Fetch ingredients for the selected recipe
    ingredients = session.query(Ingredient).join(
        RecipeIngredientsFact, Ingredient.IngredientsID == RecipeIngredientsFact.IngredientsID
    ).filter(RecipeIngredientsFact.RecipeID == recipe_id).all()

    # Display recipe details
    print(f"\nRecipe: {recipe.Title}")
    print(f"Description: {recipe.Description}")
    print(f"Cooking Time: {recipe.CookingTime} minutes")
    
       # Display ingredients
    print("Ingredients:")
    for ingredient in ingredients:
        print(f"- {ingredient.Ingredients}")

# Test the function with a specific recipe ID
test_recipe_id = 3000005  # Replace with a valid RecipeID from your database
get_recipe_details(test_recipe_id)



Fetching details for recipe ID: 3000005
Recipe details found for 3000005: Winter Squash Apple And Walnut Soup

Recipe: Winter Squash Apple And Walnut Soup
Description: 
Cooking Time: 15 minutes
Ingredients:
- Winter Squash
- Pitted Black Olives
- Dried Thyme
- Salt
- Sesame Ginger Salad Dressing
- Strawberry
- Corn Muffin Mix
- Sweet Italian Sausage
- Whipped Cream


function below works great!

In [29]:
def get_ingredients_for_recipe(recipe_id):
    ingredients = session.query(Ingredient).join(
        RecipeIngredientsFact, Ingredient.IngredientsID == RecipeIngredientsFact.IngredientsID
    ).filter(RecipeIngredientsFact.RecipeID == recipe_id).all()
    
    return ingredients

# Test the function with a specific recipe ID
test_recipe_id = 3000005  # Replace with a valid RecipeID from your database
ingredients = get_ingredients_for_recipe(test_recipe_id)

# Print the ingredients
print(f"Ingredients for recipe ID {test_recipe_id}:")
for ingredient in ingredients:
    print(f"- {ingredient.Ingredients}")



Ingredients for recipe ID 3000005:
- Winter Squash
- Pitted Black Olives
- Dried Thyme
- Salt
- Sesame Ginger Salad Dressing
- Strawberry
- Corn Muffin Mix
- Sweet Italian Sausage
- Whipped Cream


In [21]:
import sqlite3

# Connect to the SQLite database using the correct path
conn = sqlite3.connect("P:\\MABA\\Seminar\\recipes database\\my_recipes.db")
cursor = conn.cursor()

# Example SQL query using INSTR to find recipes with 'soup' in the title
cursor.execute("SELECT * FROM Recipes WHERE INSTR(Title, 'soup') > 0")
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

# Close the connection
conn.close()



(3027512, 'Kremithosoupa Onion Soup', '', 35, '', 4, '8/12/2001', 8, '')
(3067693, 'Bltsoup', '', 40, '', 15, '11/4/2004', 10, '')


testing qfunction in Squlite and not Sqlalchemy

In [23]:
import sqlite3

def get_ingredients_for_recipe(recipe_id):
    # Connect to the SQLite database
    conn = sqlite3.connect("P:\\MABA\\Seminar\\recipes database\\my_recipes.db")
    cursor = conn.cursor()

    # SQL query to get ingredients for the specified recipe
    cursor.execute("""
        SELECT Ingredients.Ingredients 
        FROM Ingredients
        JOIN Recipe_ingredients_fact_table ON Ingredients.IngredientsID = Recipe_ingredients_fact_table.IngredientsID
        WHERE Recipe_ingredients_fact_table.RecipeID = ?
    """, (recipe_id,))

    # Fetch all results
    ingredients = cursor.fetchall()

    # Close the connection
    conn.close()

    return [ingredient[0] for ingredient in ingredients]  # Return a list of ingredients

# Example usage
recipe_id = 3000005  # Replace with the desired RecipeID
ingredients = get_ingredients_for_recipe(recipe_id)

if ingredients:
    print(f"Ingredients for Recipe ID {recipe_id}:")
    for ingredient in ingredients:
        print(f"- {ingredient}")
else:
    print(f"No ingredients found for Recipe ID {recipe_id}.")


Ingredients for Recipe ID 3000005:
- Winter Squash
- Pitted Black Olives
- Dried Thyme
- Salt
- Sesame Ginger Salad Dressing
- Strawberry
- Corn Muffin Mix
- Sweet Italian Sausage
- Whipped Cream
